# Company Churn Prediction
Creating a simple Randomforest Classifier from records about companies, industry, revenue, # of employee... using SageMaker SDK

In [ ]:
import numpy as np
import pandas as pd
import boto3
import re, os
import sagemaker

In [ ]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [ ]:
BUCKET_NAME="jb-data-set"
PREFIX="churn"
MODEL_DIR="s3://jb-data-set/churn/"
MODEL_NAME="churn-scoring-model.joblib"
data_file="companies.csv"


## Build the training and validation data sets

Be sure to have the companies.csv file on local folder, or run the simulator to create data

In [ ]:
datafile_path=os.path.join("./", data_file)
df = pd.read_csv(datafile_path)
df.info()

In [ ]:
industryCat=df["Industry"].unique()
industry_type = CategoricalDtype(categories=industryCat)